In [ ]:
import requests
import nba_api
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from nba_api.stats.endpoints import playercareerstats
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import scoreboardv2
from nba_api.stats.endpoints import playbyplayv2
from datetime import datetime, timedelta
from nba_api.stats.endpoints import boxscoretraditionalv2
from time import sleep

In [ ]:
today = datetime.today()
start_date = datetime(2024, 10, 22)
date_list = [start_date + timedelta(days=x) for x in range((today - start_date).days)]

# Print the list of dates
for date in date_list:
    print(date.strftime('%Y-%m-%d'))

In [ ]:
games_df = pd.DataFrame()
# Step 1: Specify the Date (e.g., today's date)
for date in date_list:
# Step 2: Fetch Games on the Date
    try:
        scoreboard_data = scoreboardv2.ScoreboardV2(game_date=date)
        scoreboard = scoreboard_data.get_data_frames()[0]
        # Step 3: Convert to DataFrame
        games_df = pd.concat([games_df, scoreboard], ignore_index=True)
        sleep(0.5)
    except Exception as e:
        print(f"Failed to fetch data")
        
# Step 4: Extract Relevant Game Info
# print(games_df[['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_STATUS_TEXT']])


In [ ]:
games_df

In [ ]:
gameID = games_df['GAME_ID']
gameID

In [ ]:
pbp_df = pd.DataFrame()
# Example Game ID (You can get this from team game logs or live scoreboard)
for game_id in gameID:  # Replace with the desired game ID

# Fetch Play-by-Play data
#     try:
        pbp = playbyplayv2.PlayByPlayV2(game_id=game_id)
        
        # Convert to Pandas DataFrame
        stats = pbp.get_data_frames()[0]
        pbp_df = pd.concat([pbp_df, stats], ignore_index=True)
        sleep(6*10e-3)
    # except Exception as e:
    #     print(f"Failed to fetch data")
pbp_df

In [ ]:
pbp_df.columns

In [ ]:
# stats_in_game = pbp_df.to_numpy()
# np.save('stats_in_game.npy', stats_in_game)


In [ ]:
data = np.load('stats_in_game.npy', allow_pickle=True)
data

In [ ]:
data = pd.DataFrame(data, columns=pbp_df.columns)


In [ ]:
player_df = pd.DataFrame(gameID)

In [ ]:
player_df

In [ ]:
data = pd.read_csv('stats_in_game.csv')

In [ ]:
# Filter for scoring plays
scoring_plays = pbp_df[pbp_df['EVENTMSGTYPE'] == 1]

# Filter for fouls
fouls = pbp_df[pbp_df['EVENTMSGTYPE'] == 6]

# Display specific columns
print(scoring_plays[['PCTIMESTRING', 'PERIOD', 'HOMEDESCRIPTION', 'VISITORDESCRIPTION', 'SCORE']])

In [ ]:
# Convert time strings to a numerical value
data['TIME_REMAINING'] = 720 - (data['PCTIMESTRING'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1])))



In [ ]:
data.to_csv('stats_in_game.csv')

In [ ]:
data = pd.read_csv('stats_in_game.csv')
data

In [ ]:
data.columns

In [ ]:
players_df = pd.DataFrame()
for id in gameID:  # Use the game_id from your DataFrame
    boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=id)
    xl = boxscore.get_data_frames()[0]
    condition = xl['START_POSITION'] != ''
    players_df = pd.concat([players_df, (xl.loc[condition, ['GAME_ID', 'TEAM_ABBREVIATION', 'PLAYER_NAME', 'START_POSITION']])], ignore_index=True)
players_df

In [ ]:
for i in range(278):
    cond1 = players_df['GAME_ID'] == player_df['GAME_ID'][i]
    player_df.loc[i, 'TEAM_1'] = players_df.loc[cond1,'TEAM_ABBREVIATION'].unique()[0]
    player_df.loc[i, 'PLAYER_1.1'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[0]
    player_df.loc[i, 'PLAYER_1.2'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[1]
    player_df.loc[i, 'PLAYER_1.3'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[2]
    player_df.loc[i, 'PLAYER_1.4'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[3]
    player_df.loc[i, 'PLAYER_1.5'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[4]
    player_df.loc[i, 'TEAM_2'] = players_df.loc[cond1,'TEAM_ABBREVIATION'].unique()[1]
    player_df.loc[i, 'PLAYER_2.1'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[5]
    player_df.loc[i, 'PLAYER_2.2'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[6]
    player_df.loc[i, 'PLAYER_2.3'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[7]
    player_df.loc[i, 'PLAYER_2.4'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[8]
    player_df.loc[i, 'PLAYER_2.5'] = players_df.loc[cond1,'PLAYER_NAME'].unique()[9]


player_df

In [ ]:
#demo

In [ ]:
final_data = data[['GAME_ID', 'SCOREMARGIN', 'SCORE', 'TIME_REMAINING', 'PERIOD']]


In [ ]:
final_data['TEAM_1'] = None
final_data['TEAM_2'] = None
final_data['PLAYER_1.1'] = None
final_data['PLAYER_1.2'] = None
final_data['PLAYER_1.3'] = None
final_data['PLAYER_1.4'] = None
final_data['PLAYER_1.5'] = None
final_data['PLAYER_2.1'] = None
final_data['PLAYER_2.2'] = None
final_data['PLAYER_2.3'] = None
final_data['PLAYER_2.4'] = None
final_data['PLAYER_2.5'] = None

j = 0
for i in range(124992): 
        game_cond = player_df['GAME_ID'].astype(int) == final_data.loc[i, 'GAME_ID']
        final_data.loc[i, 'TEAM_1'] = player_df.loc[game_cond,'TEAM_1'].to_numpy()
        final_data.loc[i, 'PLAYER_1.1'] = player_df.loc[game_cond,'PLAYER_1.1'].to_numpy()
        final_data.loc[i, 'PLAYER_1.2'] = player_df.loc[game_cond,'PLAYER_1.2'].to_numpy()
        final_data.loc[i, 'PLAYER_1.3'] = player_df.loc[game_cond,'PLAYER_1.3'].to_numpy()
        final_data.loc[i, 'PLAYER_1.4'] = player_df.loc[game_cond,'PLAYER_1.4'].to_numpy()
        final_data.loc[i, 'PLAYER_1.5'] = player_df.loc[game_cond,'PLAYER_1.5'].to_numpy()
        final_data.loc[i, 'TEAM_2'] = player_df.loc[game_cond,'TEAM_2'].to_numpy()
        final_data.loc[i, 'PLAYER_2.1'] = player_df.loc[game_cond,'PLAYER_2.1'].to_numpy()
        final_data.loc[i, 'PLAYER_2.2'] = player_df.loc[game_cond,'PLAYER_2.2'].to_numpy()
        final_data.loc[i, 'PLAYER_2.3'] = player_df.loc[game_cond,'PLAYER_2.3'].to_numpy()
        final_data.loc[i, 'PLAYER_2.4'] = player_df.loc[game_cond,'PLAYER_2.4'].to_numpy()
        final_data.loc[i, 'PLAYER_2.5'] = player_df.loc[game_cond,'PLAYER_2.5'].to_numpy()
        if final_data.loc[i, 'GAME_ID'].astype(int) != final_data.loc[i+1, 'GAME_ID'].astype(int) and j < 277:
            j+=1
            print(j)


In [ ]:
# final_data
game_cond = player_df['GAME_ID'].astype(int) == final_data.loc[1000, 'GAME_ID']
print((player_df.loc[game_cond,'PLAYER_1.1']))
# player_df
# print(player_df.index())
final_data

In [ ]:
final_data.to_csv('final_data.csv')

In [ ]:
temp = pd.read_csv('final_data.csv')
temp

In [ ]:
# Plot the score progression
plt.plot(scoring_plays['TIME_REMAINING'], scoring_plays['HOME_SCORE'], label='Home Team')
plt.plot(scoring_plays['TIME_REMAINING'], scoring_plays['AWAY_SCORE'], label='Away Team')
plt.gca().invert_xaxis()  # Invert time to show progression
plt.xlabel('Time Remaining (seconds)')
plt.ylabel('Score')
plt.title('Score Progression')
plt.legend()
plt.show()